In [3]:
import pandas as pd

In [4]:
pontuacao_f1 = [25,18,15,12,10,8,6,4,2,1,1,0] # O penultimo numero é a melhor volta e o último numero é a pontuação do pole.
pontuacao_f1_sprint = [8,7,6,5,4,3,2,1,0,0,0,0]
jogo = "assetto"

if "f1_2022" in jogo:
    path_resultados = "Y://corrida_inscritos_2023.xls"
    opcao = 1
    if opcao == 1:
        path_output_csv = 'Y://pontuacao_corrida_dos_inscritos_2023.csv'
    elif opcao == 2:
        path_output_csv = 'Y://pontuacao_corrida_dos_membros_2023.csv'
elif "f1_2023" in jogo:
    path_resultados = "Y://corrida_inscritos_2025.xls"
    opcao = 4
    if opcao == 1:
        path_output_csv = 'Y://pontuacao_corrida_dos_inscritos_2025.csv'
    elif (opcao == 2) or (opcao == 4):
        path_output_csv = 'Y://pontuacao_corrida_dos_membros_2025.csv'
elif "f1_2024" in jogo:
    path_resultados = "Y://corrida_inscritos_2025.xls"
    opcao = 2
    if opcao == 1:
        path_output_csv = 'Y://pontuacao_corrida_dos_inscritos_2025.csv'
    elif opcao == 2:
        path_output_csv = 'Y://pontuacao_corrida_dos_membros_2025.csv'
elif "assetto" in jogo:
    path_resultados = "Y://corrida_inscritos_2025.xls"
    opcao = 5
    path_output_csv = 'Y://pontuacao_corrida_dos_membros_2025.csv'       

In [5]:
if opcao == 1:
    df_resultados = pd.read_excel(path_resultados, sheet_name="crossplay")
elif opcao == 2: 
    df_resultados = pd.read_excel(path_resultados, sheet_name="membros")
elif opcao == 3:
    pontuacao_f1 = [25,18,15,12,10,8,6,4,2,1,1,2]
    df_resultados = pd.read_excel(path_resultados, sheet_name="f2")
elif opcao == 4:
    df_resultados = pd.read_excel(path_resultados, sheet_name="membros-23")
elif opcao == 5:
    df_resultados = pd.read_excel(path_resultados, sheet_name="membros-assetto")


In [6]:
df_resultados.drop(["Index"], axis=1, inplace=True)

In [7]:
df_resultados

,Austin
0,A
1,B
2,C
3,D
4,E
5,F
6,G
7,H
8,I
9,J


In [8]:
df_pontos = pd.DataFrame(columns=["Piloto", "Pontos"])

In [9]:
for etapa in list(df_resultados.columns):
    df_etapa = pd.DataFrame(columns=["Piloto", "Pontos"])
    if (etapa.endswith("-S1")) or (etapa.endswith("-S2")): #código para Sprint (Nome da etapa seria exemplo Bahrain-S)
        df_etapa["Pontos"] = pontuacao_f1_sprint
    else:
        df_etapa["Pontos"] = pontuacao_f1
    df_etapa["Piloto"] = [driver.upper() for driver in df_resultados[etapa].values]
    df_pontos = pd.concat([df_pontos, df_etapa], ignore_index=True)

#### Gerando pontuação

In [10]:
try:
    df_pontuacao = df_pontos.groupby("Piloto").sum().sort_values(by="Pontos", ascending=False).drop("DNF", axis=0)
    df_pontuacao["Lider"] = df_pontuacao["Pontos"].values[0]
    df_pontuacao["Diferenca"] = df_pontuacao["Pontos"] - df_pontuacao["Lider"]
    df_pontuacao.drop(["Lider"], axis=1, inplace=True)
    df_pontuacao
except KeyError: #nao achou dnf
    df_pontuacao = df_pontos.groupby("Piloto").sum().sort_values(by="Pontos", ascending=False)
    df_pontuacao["Lider"] = df_pontuacao["Pontos"].values[0]
    df_pontuacao["Diferenca"] = df_pontuacao["Pontos"] - df_pontuacao["Lider"]
    df_pontuacao.drop(["Lider"], axis=1, inplace=True)
    df_pontuacao

#### Gera pontuação extendida

In [11]:
for etapa in list(df_resultados.columns):
    pilotos = [driver.upper() for driver in df_resultados[etapa].values]
    df_pontuacao[etapa] = 0
    if (etapa.endswith("-S1")) or (etapa.endswith("-S2")):
        for eachDriver, points in zip(pilotos, pontuacao_f1_sprint):
            if eachDriver != 'DNF':
                if df_pontuacao.at[eachDriver, etapa] == 0:
                    df_pontuacao.at[eachDriver, etapa] = points
                else:
                    df_pontuacao.at[eachDriver, etapa] += points
    else:
        for eachDriver, points in zip(pilotos, pontuacao_f1):
            if eachDriver != 'DNF':
                if df_pontuacao.at[eachDriver, etapa] == 0:
                    df_pontuacao.at[eachDriver, etapa] = points
                else:
                    df_pontuacao.at[eachDriver, etapa] += points

In [12]:
# Reposicionando as colunas
novasColunas = list(df_resultados.columns)
novasColunas.append("Pontos")
novasColunas.append("Diferenca")
df_pontuacao = df_pontuacao[novasColunas]
df_pontuacao[:50]

,Austin,Pontos,Diferenca
Piloto,,,
A,25,25,0
B,19,19,-6
C,15,15,-10
D,12,12,-13
E,10,10,-15
F,8,8,-17
G,6,6,-19
H,4,4,-21
I,2,2,-23


In [13]:
df_pontuacao[51:]

,Austin,Pontos,Diferenca
Piloto,,,


In [14]:
df_pontuacao[["Pontos", "Diferenca"]][:50]

,Pontos,Diferenca
Piloto,,
A,25,0
B,19,-6
C,15,-10
D,12,-13
E,10,-15
F,8,-17
G,6,-19
H,4,-21
I,2,-23


In [15]:
df_pontuacao[["Pontos", "Diferenca"]][51:]

,Pontos,Diferenca
Piloto,,


In [17]:
if opcao == 1:
    df_pontuacao.to_csv(path_output_csv)
    print("Pontuação da Corrida dos Inscritos exportada com sucesso...")
elif opcao >= 2:
    df_pontuacao.to_csv(path_output_csv)
    print("Pontuação da Corrida dos Membros exportada com sucesso...")

Pontuação da Corrida dos Membros exportada com sucesso...


#### Exportando a tabela em HTML

In [15]:
# from  datetime import datetime

# # Criar um DataFrame de exemplo (substitua isso com seus próprios dados)
# numero_de_linhas = df_pontuacao.shape[0]
# numero_de_colunas = df_pontuacao.shape[1]

# html_table = df_pontuacao.to_html(index=True)

# if opcao == 1:
#     title = "Tabela Corrida dos Inscritos"
# elif opcao == 2:
#     title = "Tabela Corrida dos Membros"

# df_pontuacao = df_pontuacao.reset_index()
# df_pontuacao.index = range(1, numero_de_linhas + 1)
# html_table = df_pontuacao.to_html(index=False, escape=False)
# data_hoje = (datetime.now()).strftime("%d/%m/%Y %H:%M:%S")

# # Criar tabela HTML com CSS incorporado
# html_code = f'''
# <html>
# <head>
#     <meta charset="latin-1"> 
#     <link rel="stylesheet" type="text/css" href="styles.css">
# </head>
# <body>
#     <h4>Atualizado em: {data_hoje}</h4>
#     {html_table}
# </body>
# </html>
# '''

# # Escrever o HTML para um arquivo
# if opcao == 1:
#     nome_arquivo = 'index_corrida_inscritos.html'
# else:
#     nome_arquivo = 'index_corrida_membros.html'
    
# with open(f'{nome_arquivo}', 'w') as file:
#     file.write(html_code)

# print(f"Arquivo HTML exportado com sucesso: {nome_arquivo}.html")

### Calculo da pontuação mensal para o gift card  
Insira o intervalo das etapas equivalente ao mês que deseja calcular a pontuação.  
Exemplo: Se as etapas 3 e 4 forem as etapas realizadas no determinado mês, coloque entre colchetes '[2, 3]'

In [18]:
if opcao >= 2:
    try:
        inicio_fim = {"f1-23": (6,8), "f1-24": (4,6), "assetto": (1,3)}
        mes = {1: "Janeiro", 2: "Fevereiro", 3: "Março", 4:"Abril",
               5: "Maio", 6:"Junho", 7: "Julho", 8: "Agosto",
               9: "Setembro", 10: "Outubro", 11: "Novembro", 12: "Dezembro"}
        if opcao == 2:
            etapa_inicio = inicio_fim.get("f1-23")[0]
            etapa_fim = inicio_fim.get("f1-23")[1]
        elif opcao == 4:
            etapa_inicio = inicio_fim.get("f1-24")[0]
            etapa_fim = inicio_fim.get("f1-24")[1]
        elif opcao == 5:
            etapa_inicio = inicio_fim.get("assetto")[0]
            etapa_fim = inicio_fim.get("assetto")[1]
        # pontuacao_f1 = [25,18,15,12,10,8,6,4,2,1,1,2]
        # df_resultados = pd.read_excel(path_resultados, sheet_name="f2")
        mes_code = 10
        df_pontos = pd.DataFrame(columns=["Piloto", "Pontos"])

        for etapa in list(df_resultados.columns[etapa_inicio-1:etapa_fim]):
            df_etapa = pd.DataFrame(columns=["Piloto", "Pontos"])
            if etapa.endswith("-S1"): #código para Sprint (Nome da etapa seria exemplo Bahrain-S)
                df_etapa["Pontos"] = pontuacao_f1_sprint
            else:
                df_etapa["Pontos"] = pontuacao_f1
            df_etapa["Piloto"] = [driver.upper() for driver in df_resultados[etapa].values]
            df_pontos = pd.concat([df_pontos, df_etapa], ignore_index=True)

        try:
            df_pontuacao = df_pontos.groupby("Piloto").sum().sort_values(by="Pontos", ascending=False).drop("DNF", axis=0)
            df_pontuacao["Lider"] = df_pontuacao["Pontos"].values[0]
            df_pontuacao["Diferenca"] = df_pontuacao["Pontos"] - df_pontuacao["Lider"]
            df_pontuacao.drop(["Lider"], axis=1, inplace=True)
            df_pontuacao
        except KeyError: #nao achou dnf
            df_pontuacao = df_pontos.groupby("Piloto").sum().sort_values(by="Pontos", ascending=False)
            df_pontuacao["Lider"] = df_pontuacao["Pontos"].values[0]
            df_pontuacao["Diferenca"] = df_pontuacao["Pontos"] - df_pontuacao["Lider"]
            df_pontuacao.drop(["Lider"], axis=1, inplace=True)
            df_pontuacao

        for etapa in list(df_resultados.columns[etapa_inicio-1:etapa_fim]):
            pilotos = [driver.upper() for driver in df_resultados[etapa].values]
            df_pontuacao[etapa] = 0
            if (etapa.endswith("-S1")) or (etapa.endswith("-S2")):
                for eachDriver, points in zip(pilotos, pontuacao_f1_sprint):
                    if eachDriver != 'DNF':
                        if df_pontuacao.at[eachDriver, etapa] == 0:
                            df_pontuacao.at[eachDriver, etapa] = points
                        else:
                            df_pontuacao.at[eachDriver, etapa] += points
            else:
                for eachDriver, points in zip(pilotos, pontuacao_f1):
                    if eachDriver != 'DNF':
                        if df_pontuacao.at[eachDriver, etapa] == 0:
                            df_pontuacao.at[eachDriver, etapa] = points
                        else:
                            df_pontuacao.at[eachDriver, etapa] += points

        # Reposicionando as colunas
        novasColunas = list(df_resultados.columns[etapa_inicio-1:etapa_fim])
        novasColunas.append("Pontos")
        novasColunas.append("Diferenca")
        df_pontuacao = df_pontuacao[novasColunas]
        print(f"Pontuação dos membros do mês de {mes.get(mes_code)}")
        print(df_pontuacao[:50])
    except IndexError:
        print("Nenhuma etapa ainda no mês de {}".format(mes.get(mes_code)))
else:
    print("Pontuação mensal só é válida para a corrida dos membros!")

Pontuação dos membros do mês de Outubro
        Austin Pontos Diferenca
Piloto                         
A           25     25         0
B           19     19        -6
C           15     15       -10
D           12     12       -13
E           10     10       -15
F            8      8       -17
G            6      6       -19
H            4      4       -21
I            2      2       -23
J            1      1       -24


#### Exportar HTML para corrida dos membros

In [17]:
# if opcao == 2:
#     from  datetime import datetime

#     # Criar um DataFrame de exemplo (substitua isso com seus próprios dados)
#     numero_de_linhas = df_pontuacao.shape[0]
#     numero_de_colunas = df_pontuacao.shape[1]

#     html_table = df_pontuacao.to_html(index=True)

#     if opcao == 1:
#         title = "Tabela Corrida dos Inscritos"
#     elif opcao == 2:
#         title = "Tabela Corrida dos Membros"

#     df_pontuacao = df_pontuacao.reset_index()
#     df_pontuacao.index = range(1, numero_de_linhas + 1)
#     html_table = df_pontuacao.to_html(index=False, escape=False)
#     data_hoje = (datetime.now()).strftime("%d/%m/%Y %H:%M:%S")

#     # Criar tabela HTML com CSS incorporado
#     html_code = f'''
#     <html>
#     <head>
#         <meta charset="latin-1">
#          <link rel="stylesheet" type="text/css" href="styles.css">
#     </head>
#     <body>
#         <h2>Pontuacao dos membros do mes de {mes.get(mes_code)}
#         <h4>Atualizado em: {data_hoje}</h4>
#         {html_table}
#     </body>
#     </html>
#     '''

#     # Escrever o HTML para um arquivo
#     if opcao == 2:
#         nome_arquivo = 'index_gift_mensal.html'

#     with open(f'{nome_arquivo}', 'w') as file:
#         file.write(html_code)

#     print("Arquivo HTML exportado com sucesso: index_gift_mensal.html")

#### Atualiza index com a corrida mais recente

In [18]:
corrida_inscritos_mais_recente_id = "1qmqoRFXDlI"

In [19]:
# html_code = f"""
# <!DOCTYPE html>
# <html>
#   <head>
# 	<meta charset="latin-1">
#     <meta name="viewport" content="width=device-width, initial-scale=1.0">
#     <title>Corrida dos Inscritos 2024</title>
# 	<link rel="stylesheet" type="text/css" href="styles.css">
#   </head>
#   <body>
  
#   <header>
#     <h1><a href='index.html'>Corrida dos Inscritos 2024</a></h1>
#   </header>
	
#     <!-- O <iframe> (e o player de vídeo) substituirão esta tag <div>. -->
#     <div id="player"></div>

#     <script>
#       var tag = document.createElement('script');
#       tag.src = "https://www.youtube.com/iframe_api";
#       var firstScriptTag = document.getElementsByTagName('script')[0];
#       firstScriptTag.parentNode.insertBefore(tag, firstScriptTag);

#       var player;
#       function onYouTubeIframeAPIReady() {{
#         player = new YT.Player('player', {{
#           height: '100%',
#           width: '100%',
#           videoId: '{corrida_inscritos_mais_recente_id}',
# 		  playerVars: {{ 'autoplay': 1, 'controls': 1 }},
#           events: {{
#             'onReady': onPlayerReady,   
#           }}
#         }});
# 		player.setPlaybackQuality('hd720')
#       }}

#       function onPlayerReady(event) {{
#         event.target.playVideo();
#       }}

#       var done = false;
#       function onPlayerStateChange(event) {{
#         if (event.data == YT.PlayerState.PLAYING && !done) {{
#           done = true;
#         }}
#       }}

#       function stopVideo() {{
#         player.stopVideo();
#       }}
	  
	  
#     </script>
#   </body>
# </html>
# """

# with open(f'ultima_corrida_inscritos.html', 'w') as file:
#         file.write(f'{html_code}')

#### Ligar o server em Python 

In [20]:
#!python -m http.server